In [ ]:
##### Extract networks using OSMnx and add elevation as a node attribute #####
##############################################################################


import os
import osmnx as ox
import geopandas as gpd
import networkx as nx

# Loop in barriers folder (original polygons, not buffers)
rootdir = r'E:\NC_CUDEM\Barriers'
extensions = ('.shp')

for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        ext = os.path.splitext(file)[-1].lower()
        if ext in extensions:
            print (file)
            file_path = os.path.join(subdir, file)

            # Read polygons, extract geometry (shapely object), and clean it with buffer
            poly = gpd.read_file(file_path)
            poly_geo = poly['geometry'].iloc[0]
            poly_geo = poly_geo.buffer(0)

            # Pull the roads with OSMnx
            G = ox.graph_from_polygon(poly_geo, network_type='drive', simplify=True, clean_periphery=True)
            print(len(G.nodes))
            
            # Give to each node a new index based on integers from 0 and then add the osmid as an attribute
            osmids = list(G.nodes)
            G = nx.relabel.convert_node_labels_to_integers(G)
            osmid_values = {k:v for k, v in zip(G.nodes, osmids)}
            nx.set_node_attributes(G, osmid_values, 'osmid')

            # Open nodes.shp with elevation data and add as an attribute to the nodes in the graph
            shp= gpd.read_file(r'E:\NC_CUDEM\Nodes\{0}'.format(file))
            print(len(shp.elevation))
            nx.set_node_attributes(G, shp.elevation, 'Elevations')

            # Save street network as graphml
            name=file.replace('.shp','')
            ox.save_graphml(G, filepath=r'E:\NC_CUDEM\Graphs\{0}.graphml'.format(name))
            
        

In [ ]:
##### Plot connected components, GCS behavior and nodes elevation - all barriers #####
######################################################################################

import os
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import geopandas as gpd
import osmnx as ox
import networkx as nx
import pandas as pd
import numpy as np
import contextily as ctx

rootdir = 'E:\\NC_CUDEM\\Graphs'
extensions = ('.graphml')

##### GCS behavior #####

# Loop through files and open barrier graphml
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        ext = os.path.splitext(file)[-1].lower()
        if ext in extensions:
            file_path = os.path.join(subdir, file)
            barrier = file.replace(".graphml","")
            print(barrier)
            G = nx.read_graphml(file_path)
            
            # Pull out elevation attribute
            E = nx.get_node_attributes(G,'Elevations')
            # Convert str values in float to be able to sort them 
            E = dict(zip(E.keys(), [float(value) for value in E.values()]))
            # Sort it based on elevation, min first
            Sorted_E = sorted(E.items(), key=lambda item: item[1])
            CCs = np.zeros([len(Sorted_E),2])
            # Select first element of each tuple in the list (nodes ID):
            FT = [i[0] for i in Sorted_E]
            # Select second element of each tuple in the list (elevation) and convert to float
            ST = [i[1] for i in Sorted_E]
            for i in range(len(ST)):
                ST[i] = float(ST[i])

            # Loop through all nodes
            for i in range(0, len(FT)):
                # Find the node with lowest elevation from the list using i and remove it
                G.remove_nodes_from(FT[0:i])
                # Check number of connected components/clusters
                CCs[int(i),0] = nx.algorithms.components.number_weakly_connected_components(G)
                # Check giant component size
                CCs[int(i),1]= largest_cc = len(max(nx.weakly_connected_components(G), key=len))/len(FT)
                        
            # Check elevation of node that causes the highest drop in the GCS
            Cliff = np.zeros([len(CCs),1])
            N= len(Cliff)
            GCS = CCs[:,1]
            V1 = GCS[:-1]  # create vector with all but the last element 
            V2 = GCS[1:]   # create vector with all but the first element
            for i in range(0,len(V1)):
                Cliff[i,0]= V1[i]-V2[i] # calculate differences in GCS between the removal of one node and the next
            m = max(Cliff) # max value in cliff corresponds to largest drop in the GCS
            pos=[i for i, j in enumerate(Cliff) if j == m] # obtain position of node that causes the drop (top of the cliff)
            pos=pos[0] # get the integer 
            elev=ST[pos] # find elevation of the node that cracks the network
            removed=pos+1 # number of nodes removed until reaching the drop (target node included)
            
            # Plot GCS behavior
            
            col=[]
            for i in range(0,len(CCs)):
                if i==pos:
                    col.append('#D53032') 
                else:
                    col.append('#4682B4')  

            
            f, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
            x_coord = 1 * np.arange(len(CCs))/len(CCs) # x axis
            ax1.plot(x_coord,CCs[:,0],':ok') # plot number of connected components
            ax1.set_ylabel("Number connected components")
            for i in range (len(CCs)):
                ax2.plot(x_coord[i],CCs[i,1],'s', markersize= 10, color=col[i], linestyle='-') # plot GCS with two colors to highlight target node
            ax3 = ax2.twinx()
            ax3.plot(x_coord,ST,':o', markersize=10, color='#9ACD32', label='Elevation') # plot elevation in GCS plot
            ax2.set_ylabel("Giant Component Size",)
            ax3.set_ylabel("Elevation")
            ax2.set_xlabel("Fraction of removed nodes") 
            ax1.set_title(barrier, fontsize=22)

            legend_elements = [Line2D([0], [0], marker='s', color='#4682B4', label='Nodes',
                                      markersize=10),
                               Line2D([0], [0], marker='s', color='#D53032', label='Target node',
                                      markersize=10),
                               Line2D([0], [0], marker='s', color='#9ACD32', label='Elevation',
                                      markersize=10),
                               ]
            ax2.legend(handles=legend_elements, loc="lower left", bbox_to_anchor=(0.1, -0.4), frameon=False, fontsize=18)
            ax2.annotate('Elevation of node that breaks the network: {0} m'.format('%.2f' %elev), xy=(0.6,-0.2), xycoords='axes fraction', fontsize=18)
            ax2.annotate('Number of nodes removed: {0}'.format(pos+1), xy=(0.6,-0.3), xycoords='axes fraction', fontsize=18)
            perc_removed=int(removed)/len(nodes)*100
            ax2.annotate('% of nodes removed: {0}'.format('%.2f' %perc_removed), xy=(0.6,-0.4), xycoords='axes fraction', fontsize=18)
            f.tight_layout()

            # Calculate robustness following Schneider's equation (2011) and include it in plot
            s= sum(CCs[:,1])
            r= s/len(FT)
            r_norm= int(round(r/0.5*100))
            ax2.annotate(r'R=%s'%(r_norm)+ '%', xy=(0.5,0.9), xycoords='axes fraction', fontsize=18)
            
            
            plt.show()
            plt.rcParams["font.size"]= 20
            plt.rcParams["figure.figsize"] = (15,15)
            
            f.savefig("E:\\NC_CUDEM\\Figures\\GCS_{0}.png".format(barrier), dpi=500, facecolor='w')
            
 #######################################################################################################################                 
##### Static maps for each network using OSM as basemap ######

            # Read polygons
            poly = gpd.read_file(r"E:\NC_CUDEM\Barriers\{0}.shp".format(barrier))
            # Extract just the geometry (shapely object) part
            poly_geo = poly['geometry'].iloc[0]
            # Clean it with a buffer
            poly_geo = poly_geo.buffer(0)
            poly_geo.is_valid
            graph = ox.graph_from_polygon(poly_geo, network_type='drive', simplify=True, clean_periphery=True)
            # Plot network
            ox.plot_graph(ox.project_graph(graph))
            # Retrieve nodes and edges as geodataframes
            nodes, edges = ox.graph_to_gdfs(graph)
            # Create an index for the geodataframe nodes
            nodes['index'] = range(0, len(nodes))

            # Convert E dict in pandas dataframe and name columns
            E = pd.DataFrame(list(E.items()),columns = ['index','elevation'])
            # Convert all columns in numerics so there are no errors when merging
            E = E.apply(pd.to_numeric)
            # Join pandas dataframe to nodes geodataframe using 'index' so that the gdf has elevation
            nodes = nodes.merge(E, on='index')
            
            # Create map
            
            def color(row):
                if row['elevation'] < elev:
                    val = "black"
                elif row['elevation']== elev:
                    val = "red"
                else:
                    val = "green"
                return val
            nodes['Color'] = nodes.apply(color, axis=1) # New column with color categories 

            def size(row):
                if row['elevation'] == elev:
                    val = 50
                else:
                    val = 30
                return val
            nodes['Size'] = nodes.apply(size, axis=1) # New column with size categories

            fig, ax = plt.subplots()
            nodes = nodes.to_crs(epsg=3857) # convert to EPSG used by basemaps
            edges = edges.to_crs(epsg=3857)
            nodes.plot(ax=ax, color=nodes.Color, markersize=nodes.Size, zorder=2, legend=True) # plot nodes
            edges.plot(ax=ax, alpha=0.2, color='black', zorder=1) # plot edges
            ctx.add_basemap(ax, zoom=14, source=ctx.providers.OpenStreetMap.Mapnik) # add basemap (OSM)
            plt.xticks(fontsize=14) # reduce fontsize of x axis
            plt.yticks(fontsize=14) # reduce fontsize of y axis
            
            legend_elements = [Line2D([0], [0], marker='o', color='black', label='Connected nodes',
                                      markerfacecolor='g', markersize=12),
                               Line2D([0], [0], marker='o', color='black', label='Disconnected nodes',
                                      markerfacecolor='b', markersize=12),
                               Line2D([0], [0], marker='o', color='black', label='Target node',
                                      markerfacecolor='r', markersize=12),
                               ] # create legend
            ax.legend(handles=legend_elements, loc='best', frameon=False)
            ax.set_title(barrier, fontsize=22)
            ax.ticklabel_format(style='plain')         
            
            plt.savefig(r'E:\NC_CUDEM\Figures\{0}.png'.format(barrier), dpi=300, facecolor='w')

In [ ]:
#### Calculate basic statistics for all barriers ####
###############################################
# import os
# import osmnx as ox
# import pandas as pd
# import geopandas as gpd


# rootdir = 'E:\\NC_CUDEM\\Barriers'
# extensions = ('.shp')

# for subdir, dirs, files in os.walk(rootdir):
#     for file in files:
#         ext = os.path.splitext(file)[-1].lower()
#         if ext in extensions:
#             file_path = os.path.join(subdir, file)
#             print(file_path)
#             # Read polygons
#             poly = gpd.read_file(file_path)
#             # Extract just the geometry (shapely object) part
#             poly_geo = poly['geometry'].iloc[0]
#             # Clean it with a buffer
#             poly_geo = poly_geo.buffer(0)
#             poly_geo.is_valid
#             # Project polygon and calculate area
#             poly_prj=ox.project_gdf(poly)
#             area=float(poly_prj.area)
#             print(area/10**6)

#             # Pull network and project it
#             G = ox.graph_from_polygon(poly_geo, network_type='drive', simplify=True, clean_periphery=True)
#             G_proj = ox.project_graph(G)

#             # show some basic stats about the network
#             stats = ox.basic_stats(G_proj, area=area, circuity_dist='euclidean')

#             # delete the no longer needed dict elements
#             del stats['streets_per_node_counts']
#             del stats['streets_per_node_proportion']

#             # load as a pandas dataframe
#             df = pd.DataFrame(pd.Series(stats, name='value'))
#             df.round(1)
#             print(df)


In [ ]:
#### Calculate statistics just one barrier ####

# import geopandas as gpd
# import osmnx as ox
# import pandas as pd


## Read polygons
# poly = gpd.read_file("E:\\NC_CUDEM\\Barriers\\NC4.shp")
# poly_prj=ox.project_gdf(poly)
# area=float(poly_prj.area)
# print(area/10**6)

# # Extract just the geometry (shapely object) part
# poly_geo = poly['geometry'].iloc[0]
# # Clean it with a buffer
# poly_geo = poly_geo.buffer(0)
# poly_geo.is_valid
# # Pull network and project it
# G = ox.graph_from_polygon(poly_geo, network_type='drive', simplify=True, clean_periphery=True)
# G_proj = ox.project_graph(G)

# # show some basic stats about the network
# stats = ox.basic_stats(G_proj, area=area, circuity_dist='euclidean')
# stats
# # delete the no longer needed dict elements
# del stats['streets_per_node_counts']
# del stats['streets_per_node_proportion']

# # load as a pandas dataframe
# df = pd.DataFrame(pd.Series(stats, name='value'))
# df.round(1)

In [ ]:
#### Plot GCS behavior - individual barriers #####
##################################################

# import numpy as np
# import networkx as nx
# import matplotlib.pyplot as plt

# import geopandas as gpd
# import numpy as np
# import osmnx as ox
# import pandas as pd

# np.set_printoptions(suppress=True) # To suppress scientific notation

# # Read graph
# G = nx.read_graphml(r"E:\NC_CUDEM\Graphs\NC18.graphml")


# # Pull out elevation attribute
# E = nx.get_node_attributes(G,'Elevations')
# # Convert str values in float to be able to sort them 
# E = dict(zip(E.keys(), [float(value) for value in E.values()]))
# # Sort it based on elevation, min first
# Sorted_E = sorted(E.items(), key=lambda item: item[1])
# CCs = np.zeros([len(Sorted_E),5])
# # Select first element of each tuple in the list (nodes ID):
# FT = [i[0] for i in Sorted_E]
# # Select second element of each tuple in the list (elevation) and convert in float
# ST = [i[1] for i in Sorted_E]
# for i in range(len(ST)):
#     ST[i] = float(ST[i])

# # Loop through all nodes
# for i in range(0, len(FT)):
#     CCs[int(i),2]= int(FT[i])
#     CCs[int(i),3]= (ST[i])
#     # Find the node with lowest elevation from the list using i and remove it
#     G.remove_nodes_from(FT[0:i])
#     # Check number of connected components/clusters
#     CCs[int(i),0] = nx.algorithms.components.number_weakly_connected_components(G)
#     # Check giant component size
#     CCs[int(i),1]= largest_cc = len(max(nx.weakly_connected_components(G), key=len))/len(FT)

# # Check elevation of node that causes the cliff

# Cliff = np.zeros([len(CCs),1])
# N= len(Cliff)

# GCS = CCs[:,1]
# V1 = GCS[:-1]  # all but the last element 
# V2 = GCS[1:]   # all but the first element

# for i in range(0,len(V1)):
#     Cliff[i,0]= V1[i]-V2[i]

# m = max(Cliff) # largest difference
# pos=[i for i, j in enumerate(Cliff) if j == m] # position of node on the top of the largest cliff
# pos=pos[0] # get the integer 
# elev=ST[pos] # find elevation of the node on the bottom of the cliff, the one that cracks the network
# print(elev) # elevation of node that breaks the network
# print(FT[pos]) # ID of the node that breaks the network
# print(pos)

# col=[]
# for i in range(0,len(CCs)):
#     if i==pos:
#         col.append('#D53032') 
#     else:
#         col.append('#4682B4')  
        

# # Plot
# x_coord = 1 * np.arange(len(CCs))/len(CCs)
# f, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
# ax1.plot(x_coord,CCs[:,0],':ok')
# ax1.set_ylabel("Number connected components")

# for i in range (len(CCs)):
#     ax2.plot(x_coord[i],CCs[i,1],'s', markersize= 10, color=col[i], linestyle='-')

# ax3 = ax2.twinx()
# ax3.plot(x_coord,ST,':o', markersize=10, color='#9ACD32', label='Elevation')


# ax2.set_ylabel("Giant Component Size",)
# ax3.set_ylabel("Elevation")
# ax2.set_xlabel("Fraction of removed nodes") 
# ax1.set_title('NC10', fontsize=22)


# legend_elements = [Line2D([0], [0], marker='s', color='#4682B4', label='Nodes',
#                           markersize=10),
#                    Line2D([0], [0], marker='s', color='#D53032', label='Disconnecting node',
#                           markersize=10),
#                    Line2D([0], [0], marker='s', color='#9ACD32', label='Elevation',
#                           markersize=10),
#                    ]
# ax2.legend(handles=legend_elements, loc="lower center", bbox_to_anchor=(0.2, -0.3), frameon=False, fontsize=10)
# ax2.annotate('Elevation of node that breaks the network: {0} m'.format('%.2f' %elev), xy=(0.6,-0.2), xycoords='axes fraction', fontsize=18)
# ax2.annotate('Number of nodes removed: {0}'.format(pos+1), xy=(0.6,-0.3), xycoords='axes fraction', fontsize=18)
# perc_removed=int(removed)/len(nodes)*100
# ax2.annotate('% of nodes removed: {0}'.format('%.2f' %perc_removed), xy=(0.6,-0.4), xycoords='axes fraction', fontsize=18)
# f.tight_layout()

# # Calculate robustness following Schneider's equation (2011)

# s= sum(CCs[:,1])
# r= s/len(FT)
# r_norm= int(round(r/0.5*100))
# textstr=r'R=%s'%(r_norm)+ '%'
# # Add text with R value
# plt.text( 0.75, 0.20, r'R=%s'%(r_norm)+ '%',
#     ha='left', va='top',
#     transform=f.transFigure
# )

# # Plot
# plt.show()
# plt.rcParams["font.size"]= 20
# plt.rcParams["figure.figsize"] = (15,15)
# #####################################################################################################################

# # Read polygons
# poly = gpd.read_file(r"E:\NC_CUDEM\Barriers\NC18.shp")
# # Extract just the geometry (shapely object) part
# poly_geo = poly['geometry'].iloc[0]
# # Clean it with a buffer
# poly_geo = poly_geo.buffer(0)
# poly_geo.is_valid
# graph = ox.graph_from_polygon(poly_geo, network_type='drive', simplify=True, clean_periphery=True)
# # Plot network
# ox.plot_graph(ox.project_graph(graph))
# # Retrieve nodes and edges
# nodes, edges = ox.graph_to_gdfs(graph)
# # Create an index for the geodataframe nodes
# nodes['index'] = range(0, len(nodes))

# # Convert dict in pandas dataframe and name columns
# E = pd.DataFrame(list(E.items()),columns = ['index','elevation'])
# # Check types of pandas dataframe or nodes dataframe to make sure 'index' has the same type in both
# # E.dtypes / nodes.dtypes
# # Convert all columns in numerics so there are no error when merging
# E = E.apply(pd.to_numeric)

# # Join pandas dataframe to nodes geodataframe using 'index' so that the gdf has elevation
# nodes = nodes.merge(E, on='index')
# nodes

# def color(row):
#     if row['elevation'] < elev:
#         val = "black"
#     elif row['elevation']== elev:
#         val = "red"
#     else:
#         val = "green"
#     return val

# nodes['Color'] = nodes.apply(color, axis=1)

# def size(row):
#     if row['elevation'] == elev:
#         val = 50
#     else:
#         val = 30
#     return val

# nodes['Size'] = nodes.apply(size, axis=1)

# import contextily as ctx

# fig, ax = plt.subplots()
# nodes = nodes.to_crs(epsg=3857)
# nodes.plot(ax=ax, color=nodes.Color, markersize=nodes.Size, zorder=2, legend=True)
# edges = edges.to_crs(epsg=3857)
# edges.plot(ax=ax, alpha=0.2, color='black', zorder=1)
# ctx.add_basemap(ax, zoom=14, source=ctx.providers.OpenStreetMap.Mapnik)
# plt.xticks(fontsize=14)
# plt.yticks(fontsize=14)

# from matplotlib.patches import Patch
# from matplotlib.lines import Line2D

# legend_elements = [Line2D([0], [0], marker='o', color='black', label='Connected nodes',
#                           markerfacecolor='g', markersize=12),
#                    Line2D([0], [0], marker='o', color='black', label='Disconnected nodes',
#                           markerfacecolor='b', markersize=12),
#                    Line2D([0], [0], marker='o', color='black', label='Node that breaks the network',
#                           markerfacecolor='r', markersize=12),
#                    ]
# ax.legend(handles=legend_elements, loc="best", frameon=False)
# ax.set_title('NC13', fontsize=22)
# ax.ticklabel_format(style='plain') 
# ax.axis('scaled')
# plt.savefig(r'E:\NC_CUDEM\Figures\NC18.png', dpi=300, facecolor='w')


In [ ]:
#### Plot nodes in an interactive map ####
##########################################

# import geopandas as gpd
# import numpy as np
# import osmnx as ox
# import pandas as pd
# import folium

# # Read polygons
# poly = gpd.read_file(r"E:\NC_CUDEM\Barriers\NC22.shp")
# # Extract just the geometry (shapely object) part
# poly_geo = poly['geometry'].iloc[0]
# # Clean it with a buffer
# poly_geo = poly_geo.buffer(0)
# poly_geo.is_valid
# G = ox.graph_from_polygon(poly_geo, network_type='drive', simplify=True, clean_periphery=True)
# # Plot network
# ox.plot_graph(ox.project_graph(G))
# # Retrieve nodes and edges
# nodes, edges = ox.graph_to_gdfs(G)
# # Create an index for the geodataframe nodes
# nodes['index'] = range(0, len(nodes))

# # Open saved graph of the same network to extract elevations
# graph = nx.read_graphml(r"E:\NC_CUDEM\Graphs\NC22.graphml")
# E = nx.get_node_attributes(graph,'Elevations')
# # Convert dict in pandas dataframe and name columns
# E = pd.DataFrame(list(E.items()),columns = ['index','elevation'])
# # Check types of pandas dataframe or nodes dataframe to make sure 'index' has the same type in both
# # E.dtypes / nodes.dtypes
# # Convert all columns in numerics so there are no error when merging
# E = E.apply(pd.to_numeric)

# # Join pandas dataframe to nodes geodataframe using 'index' so that the gdf has elevation
# nodes = nodes.merge(E, on='index')
# nodes

# # Create interactive map with folium

# m = folium.Map(location=[33.871079,-78.498601], tiles='openstreetmap', zoom_start=14, control_scale=True, prefer_canvas=True, width=750, height=750)


# # Adding a marker for each node
# for idx, node in nodes.iterrows():
#     if node['elevation'] < elev:
#         folium.Marker(location=(node['y'],node['x']),  popup=node['elevation'], icon=folium.Icon(color='black', icon='none')).add_to(m)
#     elif node['elevation'] == elev:
#         folium.Marker(location=(node['y'],node['x']), popup=node['elevation'], icon=folium.Icon(color='red', icon='none')).add_to(m)
#     else: 
#         folium.Marker(location=(node['y'],node['x']), popup=node['elevation'], icon=folium.Icon(color='green', icon='none')).add_to(m)


# # # save map to html file
# # m.save("E:\\NC_CUDEM\\map.html")
# m